In [0]:
from fastai.vision import * # import the vision module
from google.colab import drive
from google.colab import files
import pandas as pd
import numpy as np
import random
import math
import io
import os

In [0]:
def generate_random_sets(max: int, set_size: int):
    numbers = list(range(max))

    n_sets = int(math.ceil(max / set_size))

    sets = []

    for i in range(n_sets):
        k = set_size

        if set_size > len(numbers):
          k = len(numbers)

        tmp_set = random.sample(numbers, k=k)

        for j in tmp_set:
            numbers.remove(j)

        sets.append(tmp_set)

        print(i, tmp_set)

    return sets

In [0]:
vildation_indices = generate_random_sets(416, 42)

0 [410, 414, 39, 35, 270, 101, 262, 194, 55, 404, 375, 113, 312, 139, 214, 373, 225, 99, 320, 135, 365, 132, 189, 125, 254, 196, 403, 242, 248, 130, 275, 230, 70, 313, 217, 168, 266, 32, 6, 12, 88, 352]
1 [321, 51, 381, 265, 75, 133, 150, 216, 178, 203, 400, 226, 399, 114, 63, 303, 74, 83, 377, 76, 324, 126, 257, 5, 319, 109, 152, 197, 40, 383, 361, 208, 127, 384, 409, 343, 43, 282, 170, 317, 277, 72]
2 [371, 325, 272, 210, 218, 315, 290, 96, 405, 311, 129, 318, 337, 142, 46, 366, 156, 345, 368, 107, 45, 301, 408, 21, 17, 304, 376, 341, 326, 285, 372, 176, 346, 100, 94, 184, 42, 7, 141, 374, 188, 276]
3 [2, 64, 299, 370, 251, 392, 212, 47, 20, 236, 34, 160, 245, 380, 19, 271, 4, 103, 68, 195, 247, 227, 295, 249, 77, 198, 38, 162, 158, 52, 396, 328, 293, 391, 347, 357, 258, 348, 182, 16, 151, 171]
4 [402, 144, 27, 215, 333, 288, 323, 327, 79, 148, 154, 204, 261, 26, 58, 344, 300, 359, 351, 363, 356, 65, 105, 264, 134, 56, 281, 180, 104, 379, 224, 233, 329, 155, 308, 110, 10, 358, 25, 22

In [0]:
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
PATH = 'drive/My Drive/Colab Notebooks'

In [0]:
df = pd.read_csv(f'{PATH}/train_validation.csv')
df.head()

,image_name,labels
0,im0001,7
1,augmented_im0001,7
2,im0002,13 9
3,augmented_im0002,13 9
4,im0004,3


In [0]:
test_df = pd.read_csv(f'{PATH}/test_set.csv')
test = ImageList.from_df(df=test_df, path=PATH, cols=0, folder='test', suffix='.png')

In [0]:
metrics = []

for i in range(len(vildation_indices)):
  src = (
      ImageList.from_df(df=df, path=PATH, cols=0, folder='transformed', suffix='.png')
        # Load data from csv
        .split_by_idx(vildation_indices[i])
        # split data into training and validation set (15% validation)
        .label_from_df(label_delim=' ')
        # label data using the tags column (second column is default)
  )

  data = (src.transform(size=128).databunch(bs=64).normalize(imagenet_stats))

  acc_02 = partial(accuracy_thresh, thresh=0.2)
  f_score = partial(fbeta, thresh=0.2)
  learn = cnn_learner(data, models.resnet34, metrics=[acc_02, f_score])

  lr = 1e-02
  learn.fit_one_cycle(8, lr)

  print(2222, learn.validate(learn.data.valid_dl))

  learn.unfreeze()
  learn.fit_one_cycle(8, slice(1e-5, lr/5))

  # switch resolution
  data = (src.transform(tfms, size=256)
          .databunch(bs=16).normalize(imagenet_stats))
  learn.data = data

  learn.freeze()
  lr=1e-2/2
  learn.fit_one_cycle(5, slice(1e-3, 1e-2))

  learn.unfreeze()
  learn.fit_one_cycle(5, slice(1e-7, lr/5))

  metric = learn.validate(learn.data.valid_dl)

  print(i, metrics)

  metrics.append(metric)

  learn.export()

  learn = load_learner(PATH, test=test)
  preds, _ = learn.get_preds(ds_type=DatasetType.Test)

  thresh = 0.2
  labelled_preds = [' '.join([learn.data.classes[i] for i,p in enumerate(pred) if (p > thresh and i < 14)]) for pred in preds]
  fnames = test_df['image_name'].tolist()

  result_df = pd.DataFrame({'image_name':fnames, 'labels':labelled_preds}, columns=['image_name', 'labels'])
  result_df.to_csv(PATH + f'/result_cross{i}.csv', index=False)

In [0]:
for i in range(len(metrics)):
  print(i, metrics[i])

0 [0.119307816, tensor(0.9492), tensor(0.7124)]
1 [0.1251388, tensor(0.9286), tensor(0.6412)]
2 [0.19059177, tensor(0.9317), tensor(0.6132)]
3 [0.10956276, tensor(0.9476), tensor(0.7835)]
4 [0.1626016, tensor(0.9286), tensor(0.6132)]
5 [0.12193873, tensor(0.9476), tensor(0.7700)]
6 [0.14927192, tensor(0.9476), tensor(0.7212)]
7 [0.1107, tensor(0.9540), tensor(0.7850)]
8 [0.11191581, tensor(0.9508), tensor(0.7733)]
9 [0.17915815, tensor(0.9088), tensor(0.5715)]
